In [ ]:
# ## Install Coqui TTS
# ! pip install -U pip
# ! pip install TTS==0.8.0
# import os
# !git clone https://github.com/coqui-ai/TTS
# os.chdir('TTS/')

# !python3 -m pip install -r requirements.txt
# !python3 setup.py develop

# !ls


In [ ]:
!git clone https://github.com/idiap/coqui-ai-TTS
%cd coqui-ai-TTS
!pip install -e .

fatal: destination path 'coqui-ai-TTS' already exists and is not an empty directory.
/content/coqui-ai-TTS
Obtaining file:///content/coqui-ai-TTS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for coqui-tts (pyproject.toml) ... done
  Created wheel for coqui-tts: filename=coqui_tts-0.27.3-py3-none-any.whl size=13194 sha256=1ef69e9f8b3d25e46887d68319be4f3a385980f9fe30a4aed3a63cbe72930e38
  Stored in directory: /tmp/pip-ephem-wheel-cache-vlfp2904/wheels/42/68/0f/8ab4c51f62b46376e896d128de1d064635c369e8efeafd9f52
Successfully built coqui-tts
  Attempting uninstall: coqui-tts
    Found existing installation: coqui-tts 0.27.3
    Uninstalling coqui-tts-0.27.3:
      Successfully uninstalled coqui-tts-0.27.3


In [ ]:
# !nvidia-smi
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import torch
# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.models.vits import Vits, VitsAudioConfig
# BaseDatasetConfig: defines name, formatter and path of the dataset.


from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

from TTS.tts.configs.shared_configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig
#from TTS.configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig#GlowTTSConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.speakers import SpeakerManager

from TTS.config import load_config

from trainer import Trainer, TrainerArgs


In [ ]:
# # !apt install -y espeak
# !apt install -y espeak-ng
!pwd

/content/coqui-ai-TTS


In [ ]:
# !ls bn_tts

In [ ]:
import TTS
TTS.__version__

'0.27.3'

In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub
mobassir_comprehensive_bangla_tts_path = kagglehub.dataset_download('mobassir/comprehensive-bangla-tts')

print('Data source import complete.')

100%|██████████| 5.46G/5.46G [01:12<00:00, 80.5MB/s]

Extracting files...


Data source import complete.


In [ ]:
from datasets import Dataset, Audio
import pandas as pd
from pathlib import Path

# Set these to your Bangla dataset locations
# Correcting paths to use the dynamically downloaded kagglehub path
base_path = Path(mobassir_comprehensive_bangla_tts_path)
# metadata_path = base_path / "iitm_bangla_tts" / "comprehensive_bangla_tts" / "female" / "mono" / "metadata_female.txt"
# audio_dir = base_path / "iitm_bangla_tts" / "comprehensive_bangla_tts" / "female" / "mono" / "wav"

In [ ]:

output_path = "/content/tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)

male = False
pretrained = False


# Get dataset path from environment variable (passed from notebook)
mobassir_comprehensive_bangla_tts_path = os.environ.get('DATASET_PATH')
male = os.environ.get('MALE_DATASET', 'false').lower() == 'true'


pretrained_path = ''
if(pretrained):
    pretrained_path = ''
if(male):
    meta_file = base_path / "iitm_bangla_tts" / "comprehensive_bangla_tts" / "male" / "mono" / "metadata_male.txt"
    root_path = base_path / "iitm_bangla_tts" / "comprehensive_bangla_tts" / "male" / "mono"
else:
    meta_file = base_path / "iitm_bangla_tts" / "comprehensive_bangla_tts" / "female" / "mono" / "metadata_female.txt"
    root_path = base_path / "iitm_bangla_tts" / "comprehensive_bangla_tts" / "female" / "mono"



def formatter(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the LJSpeech meta data file to TTS format
    https://keithito.com/LJ-Speech-Dataset/"""
    txt_file = meta_file
    items = []
    speaker_name = "ljspeech"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wav", cols[0] + ".wav")
            try:
                text = cols[1]
            except:
                print("not found")

            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


dataset_config = BaseDatasetConfig(
     meta_file_train=meta_file,
    path=os.path.join(root_path, "")
)


In [ ]:
dataset_config

BaseDatasetConfig(formatter='', dataset_name='', path='/root/.cache/kagglehub/datasets/mobassir/comprehensive-bangla-tts/versions/3/iitm_bangla_tts/comprehensive_bangla_tts/female/mono/', meta_file_train=PosixPath('/root/.cache/kagglehub/datasets/mobassir/comprehensive-bangla-tts/versions/3/iitm_bangla_tts/comprehensive_bangla_tts/female/mono/metadata_female.txt'), ignored_speakers=None, language='', phonemizer='', meta_file_val='', meta_file_attn_mask='')

In [ ]:
# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config,formatter=formatter, eval_split=True)

In [ ]:
print(len(train_samples),len(eval_samples))

6599 66


In [ ]:
train_samples[0]

{'text': 'দীর্ঘস্থায়ী অবসাদের ক্ষেত্রে, জীবনযাপনের প্রতিটি স্তরে, হতাশার লক্ষণ দেখা যায়\n',
 'audio_file': '/root/.cache/kagglehub/datasets/mobassir/comprehensive-bangla-tts/versions/3/iitm_bangla_tts/comprehensive_bangla_tts/female/mono/wav/train_bengalifemale_01282.wav',
 'speaker_name': 'ljspeech',
 'root_path': '/root/.cache/kagglehub/datasets/mobassir/comprehensive-bangla-tts/versions/3/iitm_bangla_tts/comprehensive_bangla_tts/female/mono/',
 'language': '',
 'audio_unique_name': '#wav/train_bengalifemale_01282'}

In [ ]:
# we use the same path as this script as our training folder.
output_path = 'content/tts_model'

# audio_config = BaseAudioConfig(
#      sample_rate = 22050,
#     resample =True
# )

audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)


if(male):
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "তট৫ভিঐঋখঊড়ইজমএেঘঙসীঢ়হঞ‘ঈকণ৬ঁৗশঢঠ‌১্২৮দৃঔগও—ছউংবৈঝাযফ‍চরষঅৌৎথড়৪ধ০ুূ৩আঃপয়’নলো",
    punctuations = "-!,|.? ",
    )
else:
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "ইগং়’ুঃন১ঝূও‘ঊোছপফৈ৮ষযৎঢঈকঠিজ০৬ীটডএঅঋধচে২৩ণউয়ঢ়খলভৗসহ্ড়দথবঔাঞশরৌম—ঐআৃঘঙ‌ঁ৪৫ত",
    punctuations = ".?-!|, ",
    )


# VitsConfig: all model related values for training, validating and testing.

config = VitsConfig(
    audio=audio_config,
    run_name="vits_4_nov",
    batch_size=8, # Reduced batch size
    eval_batch_size=4, # Reduced eval batch size
    batch_group_size=0,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1,
    text_cleaner=None, #"collapse_whitespace"
    use_phonemes=False,
    compute_input_seq_cache=True,
    print_step=100,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    characters = characters_config,
    save_step=1000,
    cudnn_benchmark=True,
    test_sentences = [
        'হয়,হয়ে,ওয়া,হয়েছ,হয়েছে,দিয়ে,যায়,দায়,নিশ্চয়,আয়,ভয়,নয়,আয়াত,নিয়ে,হয়েছে,দিয়েছ,রয়ে,রয়েছ,রয়েছে।',
        'দেয়,দেওয়া,বিষয়,হয়,হওয়া,সম্প্রদায়,সময়,হয়েছি,দিয়েছি,হয়,হয়েছিল,বিষয়ে,নয়,কিয়াম,ইয়া,দেয়া,দিয়েছে,আয়াতে,দয়া।',
        'ইয়াহুদ,নয়,ব্যয়,ইয়াহুদী,নেওয়া,উভয়ে,যায়,হয়েছিল,প্রয়োজন।'
    ]
)


In [ ]:

ap = AudioProcessor.init_from_config(config)
# ap.resample = True

In [ ]:
ap.resample

False

In [ ]:

tokenizer, config = TTSTokenizer.init_from_config(config)

In [ ]:

model = Vits(config, ap, tokenizer, speaker_manager=None)

In [ ]:
# checkpoint = torch.load('/home/ansary/Shabab/vits_20_october/best_model_311129.pth')
# model.load_state_dict(checkpoint["model"])

model

Vits(
  (text_encoder): TextEncoder(
    (emb): Embedding(86, 192)
    (encoder): RelativePositionTransformer(
      (dropout): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0-5): 6 x RelativePositionMultiHeadAttention(
          (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (norm_layers_1): ModuleList(
        (0-5): 6 x LayerNorm2()
      )
      (ffn_layers): ModuleList(
        (0-5): 6 x FeedForwardNetwork(
          (conv_1): Conv1d(192, 768, kernel_size=(3,), stride=(1,))
          (conv_2): Conv1d(768, 192, kernel_size=(3,), stride=(1,))
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (norm_layers_2): ModuleList(
        (0-5): 6 x LayerNor

In [ ]:
import torch

# Clear CUDA cache to free up memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# use continue_path argument below for retraining model from last saved weight https://github.com/coqui-ai/Trainer/blob/main/trainer/trainer.py#L210

trainer = Trainer(
    TrainerArgs(continue_path = pretrained_path), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

/usr/local/lib/python3.12/dist-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: True
 | > Torch TF32 MatMul: False
 > Start 

In [ ]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.6 MB/s eta 0:00:00


In [ ]:
%%time
trainer.fit()


 > EPOCH: 0/0
 --> content/tts_model/vits_4_nov-January-18-2026_12+29PM-12b399fc
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(

 > TRAINING (2026-01-18 12:30:34) 





   --> TIME: 2026-01-18 12:30:45 -- STEP: 0/824 -- GLOBAL_STEP: 0
     | > current_lr_0: 0.0002  (0.0002)
     | > current_lr_1: 0.0002  (0.0002)
     | > loss_disc: 6.018164157867432  (6.018164157867432)
     | > loss_disc_real_0: 1.011553406715393  (1.011553406715393)
     | > loss_disc_real_1: 1.0066626071929932  (1.0066626071929932)
     | > loss_disc_real_2: 1.0265421867370605  (1.0265421867370605)
     | > loss_d

CPU times: user 21min 37s, sys: 43.8 s, total: 22min 21s
Wall time: 23min 16s


In [ ]:
!pip install noisereduce
import noisereduce as nr
from scipy.signal import butter, filtfilt

def enhance_audio(wav, sr=22050):
    # 1. Noise reduction
    wav = nr.reduce_noise(y=wav, sr=sr, stationary=True, prop_decrease=0.75)

    # 2. Dynamic range compression (simple compression)
    threshold = 0.5
    ratio = 4.0
    wav = np.where(np.abs(wav) > threshold, threshold + (np.abs(wav) - threshold) / ratio * np.sign(wav), wav)

    # 3. Final normalization
    if np.max(np.abs(wav)) > 0:
        wav = wav / np.max(np.abs(wav)) * 0.95

    # 4. High-pass filter (remove DC and low freq noise)
    def butter_highpass(cutoff, fs, order=5):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='high', analog=False)
        return b, a

    b, a = butter_highpass(80, sr, order=5)
    wav = filtfilt(b, a, wav)

    return wav

In [ ]:
import os
import glob
import time
from TTS.utils.synthesizer import Synthesizer
from IPython.display import Audio, display

# Paths
model_dir = "/content/coqui-ai-TTS/content/tts_model/vits_4_nov-January-18-2026_12+29PM-12b399fc"
config_dir = "/content/coqui-ai-TTS/content/tts_model/vits_4_nov-January-18-2026_12+29PM-12b399fc"
output_dir = "/content/working/output"
os.makedirs(output_dir, exist_ok=True)

# Find checkpoints and config
ckpts = sorted(glob.glob(os.path.join(model_dir, "*.pth")))
configs = sorted(glob.glob(os.path.join(config_dir, "*.json")))

print("Found checkpoints:", ckpts)
print("Found configs:", configs)

if not ckpts or not configs:
    raise RuntimeError("❌ No trained model or config found!")

# Prefer best_model if exists
best_ckpt = [c for c in ckpts if "best_model" in c]
model_path = best_ckpt[0] if best_ckpt else ckpts[-1]
config_path = configs[0]

print(f"\n✅ Loading model: {model_path}")
print(f"✅ Loading config: {config_path}")

# Init synthesizer
synthesizer = Synthesizer(
    tts_checkpoint=model_path,
    tts_config_path=config_path,
    use_cuda=True
)

# Bengali text
text = "আমরা আপনাদের জন্য নিয়ে এলাম ফ্রি এবং ওপেনসোর্স বাংলা ফন্টের সবচেয়ে বড় সংগ্রহশালা, অক্ষর ৫২! "

print(f"\n🗣 Generating speech for: {text}")

start_time = time.time()
wav = synthesizer.tts(text)
wav = enhance_audio(wav)
generation_time = time.time() - start_time

# Save wav
output_file = os.path.join(output_dir, "bengali_glowtts.wav")
synthesizer.save_wav(wav, output_file)

duration = len(wav) / synthesizer.output_sample_rate

print(f"✅ Saved to: {output_file}")
print(f"⏱️ Generation time: {generation_time:.3f} sec")
print(f"🎧 Audio duration: {duration:.2f} sec")
print(f"⚡ RTF: {generation_time / duration:.2f}x")

display(Audio(output_file))

Found checkpoints: ['/content/coqui-ai-TTS/content/tts_model/vits_4_nov-January-18-2026_12+29PM-12b399fc/best_model.pth', '/content/coqui-ai-TTS/content/tts_model/vits_4_nov-January-18-2026_12+29PM-12b399fc/best_model_825.pth']
Found configs: ['/content/coqui-ai-TTS/content/tts_model/vits_4_nov-January-18-2026_12+29PM-12b399fc/config.json']

✅ Loading model: /content/coqui-ai-TTS/content/tts_model/vits_4_nov-January-18-2026_12+29PM-12b399fc/best_model.pth
✅ Loading config: /content/coqui-ai-TTS/content/tts_model/vits_4_nov-January-18-2026_12+29PM-12b399fc/config.json

🗣 Generating speech for: আমরা আপনাদের জন্য নিয়ে এলাম ফ্রি এবং ওপেনসোর্স বাংলা ফন্টের সবচেয়ে বড় সংগ্রহশালা, অক্ষর ৫২! 
✅ Saved to: /content/working/output/bengali_glowtts.wav
⏱️ Generation time: 5.461 sec
🎧 Audio duration: 10.93 sec
⚡ RTF: 0.50x


In [ ]:
!nvidia-smi

Sun Jan 18 12:52:27 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P0             32W /   70W |    1458MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----